In [1]:
# importing the libraries
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [2]:
# setting the constants
IMAGE_SIZE = [28, 28]
BATCH_SIZE = 128
INPUT_SHAPE = (28, 28, 1)
TRAIN_RATIO = 0.75

NUM_EPOCHS = 5

In [3]:
# creating a function to split train dataset into train and validation
def _split_train_val(train_data, train_ratio=0.8):
    # getting the size
    image_count = train_data.cardinality().numpy()

    # shuffling the data
    train_data = train_data.shuffle(buffer_size=10000)

    # splitting
    dataset_train = train_data.take(image_count * train_ratio)
    dataset_validation = train_data.skip(image_count * train_ratio)
    
    return dataset_train, dataset_validation

# to rescale and resize the images
def _resize_scale_image(image, label, image_size=IMAGE_SIZE):
    image = tf.image.resize(image, image_size)
    image = image/255.
    return image, label

def get_datasets():
    # loading the dataset
    train_data, test_data = tfds.load(name="mnist", split=("train", "test"), as_supervised=True)
    
    dataset_train, dataset_validation = _split_train_val(train_data, train_ratio=TRAIN_RATIO)
    dataset_test = test_data

    print("Number of samples in train: ", dataset_train.cardinality().numpy())
    print("Number of samples in validation: ", dataset_validation.cardinality().numpy())
    print("Number of samples in test: ", dataset_test.cardinality().numpy())

    # applying the rescale function
    dataset_train = dataset_train.map(_resize_scale_image)
    dataset_validation = dataset_validation.map(_resize_scale_image)

    # creating batches and prefetches
    dataset_train = dataset_train.batch(BATCH_SIZE)
    dataset_train = dataset_train.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    dataset_validation = dataset_validation.batch(BATCH_SIZE)
    dataset_validation = dataset_validation.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return dataset_train, dataset_validation, dataset_test

training_set, validation_set, testing_set = get_datasets()

Number of samples in train:  45000
Number of samples in validation:  15000
Number of samples in test:  10000


In [4]:
# creating the model
def get_model():
    # defining the model
    model = tf.keras.Sequential([
                                tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu", input_shape=INPUT_SHAPE, name="image"),
                                tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
                                tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu"),
                                tf.keras.layers.MaxPool2D(pool_size=(2,2)),
                                tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation="relu"),
                                tf.keras.layers.MaxPool2D(pool_size=(2,2)),
                                tf.keras.layers.GlobalAveragePooling2D(),
                                tf.keras.layers.Dense(10, activation="softmax", name="softmax_output")
    ])

    print(model.summary())

    return model

model = get_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (Conv2D)              (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d (Conv2D)             (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 3, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 1, 1, 128)        0

In [5]:
# compiling and training the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

history = model.fit(training_set, 
          validation_data = validation_set, 
          epochs=NUM_EPOCHS)

Epoch 1/5
352/352 [==============================] - 17s 41ms/step - loss: 0.3521 - accuracy: 0.8955 - val_loss: 0.1402 - val_accuracy: 0.9570
Epoch 2/5
352/352 [==============================] - 5s 15ms/step - loss: 0.1014 - accuracy: 0.9690 - val_loss: 0.0840 - val_accuracy: 0.9734
Epoch 3/5
352/352 [==============================] - 5s 15ms/step - loss: 0.0720 - accuracy: 0.9781 - val_loss: 0.0851 - val_accuracy: 0.9755
Epoch 4/5
352/352 [==============================] - 5s 15ms/step - loss: 0.0605 - accuracy: 0.9807 - val_loss: 0.0602 - val_accuracy: 0.9813
Epoch 5/5
352/352 [==============================] - 5s 15ms/step - loss: 0.0487 - accuracy: 0.9842 - val_loss: 0.0469 - val_accuracy: 0.9857


In [6]:
# checking the performance on the test set
results = model.evaluate(testing_set.batch(BATCH_SIZE), verbose=2)
for name, value in zip(model.metrics_names, results):
    print(f"{name}: {round(value, 3)}")

79/79 - 1s - loss: 13.9706 - accuracy: 0.9751 - 1s/epoch - 18ms/step
loss: 13.971
accuracy: 0.975


In [7]:
# getting some images from the test set to visualize the raw predictions
images = []
labels = []
for image, label in testing_set.take(10):
    images.append(image.numpy())
    labels.append(label.numpy())

softmax_result = model.predict(x=tf.constant(images))
print(softmax_result)

[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [8]:
# exporting the model
import os
import datetime
import shutil
from pathlib import Path

export_path = Path("models")
model_path = export_path / "default" / "mnist_{}".format(datetime.datetime.now().strftime("%Y%m%d_%H%M%S"))
model.save(model_path)

INFO:tensorflow:Assets written to: models/default/mnist_20220112_135012/assets


INFO:tensorflow:Assets written to: models/default/mnist_20220112_135012/assets


In [9]:
# lets see the files in the saved models
!find models/default

models/default
models/default/mnist_20220112_133734
models/default/mnist_20220112_133734/saved_model.pb
models/default/mnist_20220112_133734/variables
models/default/mnist_20220112_133734/variables/variables.index
models/default/mnist_20220112_133734/variables/variables.data-00000-of-00001
models/default/mnist_20220112_133734/keras_metadata.pb
models/default/mnist_20220112_133734/assets
models/default/mnist_20220112_135012
models/default/mnist_20220112_135012/saved_model.pb
models/default/mnist_20220112_135012/variables
models/default/mnist_20220112_135012/variables/variables.index
models/default/mnist_20220112_135012/variables/variables.data-00000-of-00001
models/default/mnist_20220112_135012/keras_metadata.pb
models/default/mnist_20220112_135012/assets


In [10]:
# lets see the prediction meta-data about the trained model
!saved_model_cli show --dir {model_path} --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['image_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 28, 28, 1)
      name: serving_default_image_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['softmax_output'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [11]:
# we can see that it takes an image and returns the softmax score for the ten classes
restored = tf.keras.models.load_model(model_path)
infer = restored.signatures["serving_default"]
outputs = infer(image_input=tf.constant(images, dtype=tf.float32))
softmax_outputs = outputs["softmax_output"]
print(softmax_outputs, end="\n\n")

# if we want the final results, 
# we would need to convert the results in to argmax for each result
print(tf.math.argmax(softmax_outputs, axis=1).numpy())

tf.Tensor(
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(10, 10), dtype=float32)

[2 0 4 8 7 6 0 6 3 1]


In [12]:
# now considering we are serving two different clients,
# one may require the final results while the other may require the softmax results as well.

# creating the default signature which outputs only the final classes
@tf.function(input_signature=[tf.TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32)])
def final_classes(images):
    softmax_outputs = model(images, training=False)
    final_classes = tf.math.argmax(softmax_outputs, axis=1)
    return {
        "outputs": final_classes
    }

# the secondary signature that returns both outputs
@tf.function(input_signature=[tf.TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32)])
def add_final_classes(images):
    softmax_outputs = model(images, training=False)
    final_classes = tf.math.argmax(softmax_outputs, axis=1)
    return {
        "softmax_outputs": softmax_outputs,
        "outputs": final_classes
    }

shutil.rmtree(export_path / "final_classes", ignore_errors=True)
final_export_path = export_path / "final_classes" / "mnist_{}".format(datetime.datetime.now().strftime("%Y%m%d_%H%M%S"))
model.save(final_export_path, signatures={"serving_default": final_classes, "serving_with_softmax": add_final_classes})

INFO:tensorflow:Assets written to: models/final_classes/mnist_20220112_135018/assets


INFO:tensorflow:Assets written to: models/final_classes/mnist_20220112_135018/assets


In [13]:
# we can now see that the default signature would give only the final classes
!saved_model_cli show --dir {final_export_path} --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['images'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 28, 28, 1)
      name: serving_default_images:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['outputs'] tensor_info:
      dtype: DT_INT64
      shape: (-1)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [14]:
# we can now see that the secondary signature would give both the outputs
!saved_model_cli show --dir {final_export_path} --tag_set serve --signature_def serving_with_softmax

The given SavedModel SignatureDef contains the following input(s):
  inputs['images'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 28, 28, 1)
      name: serving_with_softmax_images:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['outputs'] tensor_info:
      dtype: DT_INT64
      shape: (-1)
      name: StatefulPartitionedCall_1:0
  outputs['softmax_outputs'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall_1:1
Method name is: tensorflow/serving/predict


In [15]:
# serving 1 (only final classes)
print(".: Serving serving_default :.\n")
restored = tf.keras.models.load_model(final_export_path)
infer = restored.signatures["serving_default"]
outputs = infer(images=tf.constant(images, dtype=tf.float32))
outputs = outputs["outputs"]
print(outputs.numpy(), end="\n\n")

# serving 2 (with softmax outputs)
print(".: Serving serving_with_softmax :.\n")
restored = tf.keras.models.load_model(final_export_path)
infer = restored.signatures["serving_with_softmax"]
outputs = infer(images=tf.constant(images, dtype=tf.float32))
outputs_softmax = outputs["softmax_outputs"]
outputs = outputs["outputs"]
print(outputs_softmax.numpy(), end="\n\n")
print(outputs.numpy())

.: Serving serving_default :.

[2 0 4 8 7 6 0 6 3 1]

.: Serving serving_with_softmax :.

[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]

[2 0 4 8 7 6 0 6 3 1]
